In [ ]:
import sys, os
if os.path.abspath(os.pardir) not in sys.path:
    sys.path.insert(1, os.path.abspath(os.pardir))
import CONFIG

%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
from PIL import Image, ImageDraw
from glob import glob
import re
from google.colab.patches import cv2_imshow

In [ ]:
INPUT_DIR = CONFIG.CFG.DATA.BASE
IMAGES_OUT = CONFIG.CFG.DATA.BASE + "/new_train"

In [ ]:
train_df = pd.read_csv(os.path.join(INPUT_DIR, "train.csv"))
print(train_df.shape)
train_df.head()

In [ ]:
def resize(image_pil, width, height):
    '''
    Resize PIL image keeping ratio and using white background.
    '''
    ratio_w = width / image_pil.width
    ratio_h = height / image_pil.height
    if ratio_w < ratio_h:
        # It must be fixed by width
        resize_width = width
        resize_height = round(ratio_w * image_pil.height)
    else:
        # Fixed by height
        resize_width = round(ratio_h * image_pil.width)
        resize_height = height
    image_resize = image_pil.resize((resize_width, resize_height), Image.ANTIALIAS)
    background = Image.new('RGBA', (width, height), (255, 255, 255, 255))
    offset = (round((width - resize_width) / 2), round((height - resize_height) / 2))
    background.paste(image_resize, offset)
    return background.convert('RGB')

In [ ]:
def parse_bbox(bbox_str):
    replace_brac = re.sub("\[|\]", "", bbox_str)
    bbox_list = np.array([int(val[:-2]) for val in replace_brac.split(",") if val != ''])
    return bbox_list

In [ ]:
if not os.path.exists(IMAGES_OUT):
    os.makedirs(IMAGES_OUT)

In [ ]:
for i, row in train_df.iterrows():
    try:
        bbox = parse_bbox(row.bbox)
        img = Image.open(os.path.join(INPUT_DIR, "train", f"{row.image_id}.jpg"))
        cropped_img = img.crop((bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]))
        final_img = resize_with_pad(cropped_img, 250, 250)
        final_img.save(os.path.join(IMAGES_OUT, f"{row.image_id}_{i}.jpg"))
    except Exception as e:
        print(e)

In [ ]:
f"{len(glob(IMAGES_OUT+'/*.jpg'))} images generated"

In [ ]:
plt.rcParams["figure.figsize"] = (10, 10)

In [ ]:
uniq_imgs = train_df.image_id.unique()
for sample_img in uniq_imgs:
    img = cv2.imread(os.path.join(INPUT_DIR, "train", f"{sample_img}.jpg"), -1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    bboxes = train_df[train_df.image_id == sample_img].bbox
    for box in bboxes:
        try:
            bbox = parse_bbox(box)
            cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[0]+bbox[2], bbox[1]+bbox[3]), (255,255,255), -1)
        except Exception as e:
            print(e)
    for i in range(len(bboxes)):
        random_sample = (np.random.randint(1024-100), np.random.randint(1024-100))
        img1 = img[random_sample[1]:random_sample[1]+150, random_sample[0]:random_sample[0]+150]
        img2 = Image.fromarray(img1)
        final_img = resize_with_pad(img2, 250, 250)
        final_img.save(os.path.join(IMAGES_OUT, f"{sample_img}-{i}.jpg"))
    break